In [1]:
import pickle
import math

import torch
import numpy as np
from veccs import orderings

from gpytorch.kernels import MaternKernel
from sklearn.gaussian_process import kernels
from matplotlib import pyplot as plt

from batram.legmods import Data, SimpleTM
from batram.shrinkmods import ShrinkTM, EstimableShrinkTM

%load_ext autoreload
%autoreload 2

### Reading data

In [5]:

with open("../tests/data/prec_days.pkl", "rb") as f:
    prec_days = pickle.load(f)

lat = prec_days["lat"]
lon = prec_days["lon"]
obs = np.log(prec_days["precs"][:,:,0].T + 1e-10)
obs_orig = obs.copy()
locs = np.loadtxt("../tests/data/americas_locs.csv", skiprows=1, delimiter=",")
lat = locs[:, 1]
lon = locs[:, 0]

In [6]:
#newline
l=(lon/360)*2*math.pi; L=(lat/360)*2*math.pi
locs=np.vstack([np.cos(L)*np.cos(l),np.cos(L)*np.sin(l),np.sin(L)]).T

In [9]:
with open("../tests/data/americas_ordering.pkl", "rb") as f:
    data = pickle.load(f)

In [8]:
#newline
obsmean = np.mean(obs, axis=0, keepdims=True)
obdsd = np.std(obs, axis=0, keepdims=True)

### Standardizing

In [9]:
#newline
obs = (obs - obsmean) / obdsd

### Ordering

In [11]:
torch.manual_seed(6132024)
#locsorder = orderings.maxmin_cpp(locs)
locsorder = np.intp(data["locsorder"])
locsTrain = locs[locsorder, :]

In [12]:
#nn = orderings.find_nns_l2(locsTrain, 30)
nn = np.intp(data["nn"])
shuffle = torch.randperm(obs.shape[0])

obsTrain = torch.from_numpy((obs[shuffle,:])[:, locsorder]).float()

### Training

In [13]:
nsteps = 200
nTrain = [1, 3, 5, 10, 20, 30, 50, 80]

In [14]:
logscores_tm = []
logscores_shrink = []
numExpReps = 10
tm_models = []
shrink_models = []
numTestSamples = 50
ModifNumTestSampleslist = []

In [15]:
MasterIndices = np.arange(obsTrain.shape[0])
for n in nTrain:
    
    ActNumTestSamples = min(numTestSamples, (obsTrain.shape[0] - n))
    logScore_tm = torch.zeros(numExpReps)
    logScore_shrink = torch.zeros(numExpReps)
    ModifNumTestSamples = (ActNumTestSamples * torch.ones(numExpReps))
    for i in range(numExpReps):
        indices = np.random.choice(MasterIndices, n, replace=False)
        obsTrainSub = obsTrain[indices, :]
        if obsTrainSub.dim() == 1:
            obsTrainSub = obsTrainSub.unsqueeze(0)

        data = Data.new(torch.as_tensor(locsTrain).float(), obsTrainSub, torch.as_tensor(nn))

        tm = SimpleTM(data, theta_init=None, linear=False, smooth=1.5, nug_mult=4.0)
        opt = torch.optim.Adam(tm.parameters(), lr=0.01)
        sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, nsteps)
        res = tm.fit(
            nsteps, 0.1, test_data=tm.data, optimizer=opt, scheduler=sched, batch_size=900
        )

        shrink_tm = EstimableShrinkTM(data=data, linear=False, 
                            transportmap_smooth=1.5, 
                            parametric_kernel= "exponential",
                            param_nu=0.5,
                            param_ls=0.6,
                            nug_mult_bounded=False)
        opt2 = torch.optim.Adam(shrink_tm.parameters(), lr=0.01)
        sched2 = torch.optim.lr_scheduler.CosineAnnealingLR(opt2, nsteps)
        res2 = shrink_tm.fit(
            nsteps, 0.1, test_data=shrink_tm.data, optimizer=opt2, scheduler=sched2, batch_size=900,

        )
        tm_models.append(tm)
        shrink_models.append(shrink_tm)
        TestIndices = np.setdiff1d(MasterIndices, indices)
        for _j in range(ActNumTestSamples):
            try:
                with torch.no_grad():
                    logScore_tm[i] += tm.score(obsTrain[(TestIndices[_j]), :])
                logScore_shrink[i] += shrink_tm.score(obsTrain[(TestIndices[_j]), :])
            except:
                ModifNumTestSamples[i] -= 1
                continue
        print(f"{i}th experiment with {n} training samples done.")
    ModifNumTestSampleslist.append(ModifNumTestSamples)
    logscores_tm.append(logScore_tm)
    logscores_shrink.append(logScore_shrink)
        

Train Loss: -1698.097, Test Loss: -1706.705: 100%|██████████| 200/200 [00:56<00:00,  3.55it/s]


0th experiment with 1 training samples done.


Train Loss: -1595.475, Test Loss: -1591.940: 100%|██████████| 200/200 [00:45<00:00,  4.42it/s]


1th experiment with 1 training samples done.


Train Loss: -959.319, Test Loss: -966.726: 100%|██████████| 200/200 [00:48<00:00,  4.12it/s]


...

7th experiment with 80 training samples done.


Train Loss: -127388.628, Test Loss: -127366.734: 100%|██████████| 200/200 [04:20<00:00,  1.30s/it]


8th experiment with 80 training samples done.


Train Loss: -128654.302, Test Loss: -128557.133: 100%|██████████| 200/200 [04:41<00:00,  1.41s/it]


9th experiment with 80 training samples done.


In [18]:
torch.save({
 'logscores_tm': logscores_tm,
 'logscores_shrink': logscores_shrink,
 'ModifNumTestSampleslist': ModifNumTestSampleslist,
 'nTrain': nTrain,
}, '../results/dup_precipresults_americas.pt')

In [19]:
precipmodels = {
    'tm_models': tm_models,
 'shrink_models': shrink_models,  
}

with open('../results/dup_precipmodels_americas.pkl', 'wb') as f:
    pickle.dump(precipmodels, f)